In [27]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

df2 = pd.read_csv("../data/game_stats.csv")
df2.head()

def process_third(x):
    if(str(x).split('-')[-1] == '0'): return 1
    else: return float(str(x).split('-')[0])/float(str(x).split('-')[1])

stats = df2[['id', 'home.school','home.points', 'home.thirdDownEff','home.completionAttempts', 'home.yardsPerPass','home.yardsPerRushAttempt','home.totalPenaltiesYards','home.turnovers','home.possessionTime','away.school','away.points','away.thirdDownEff','away.completionAttempts', 'away.yardsPerPass','away.yardsPerRushAttempt','away.totalPenaltiesYards','away.turnovers','away.possessionTime','home.rushingAttempts','away.rushingAttempts','Year','Week']]
stats.dropna(inplace = True)
stats.loc[:,'home.pointDiff'] = df2['home.points'] - df2['away.points']
stats.loc[:,'away.pointDiff'] = -stats['home.pointDiff']
stats.loc[:,'home.completionAttempts'] = df2['home.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'away.completionAttempts'] = df2['away.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'home.yardsAllowedPerPlay'] = df2['away.totalYards']/(stats['away.completionAttempts'] + df2['away.rushingAttempts'])
stats.loc[:,'away.yardsAllowedPerPlay'] = df2['home.totalYards']/(stats['home.completionAttempts'] + df2['home.rushingAttempts'])
stats.loc[:,'home.yards_per_play'] = stats['away.yardsAllowedPerPlay']
stats.loc[:,'away.yards_per_play'] = stats['home.yardsAllowedPerPlay']
stats.loc[:,'home.forcedTO'] = df2['away.turnovers']
stats.loc[:,'away.forcedTO'] = df2['home.turnovers']
stats.loc[:,'home.passAttemptsAllowed'] = stats['away.completionAttempts']
stats.loc[:,'away.passAttemptsAllowed'] = stats['home.completionAttempts']
stats.loc[:,'home.yardsAllowedPerRush'] = df2['away.yardsPerRushAttempt']
stats.loc[:,'away.yardsAllowedPerRush'] = df2['home.yardsPerRushAttempt']
stats.loc[:,'home.thirdDownEff'] = stats.loc[:,'home.thirdDownEff'].apply(process_third)
stats.loc[:,'away.thirdDownEff'] = stats.loc[:,'away.thirdDownEff'].apply(process_third)
stats.loc[:,'home.totalPenaltiesYards'] = stats.loc[:,'home.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'away.totalPenaltiesYards'] = stats.loc[:,'away.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'home.possessionTime'] = stats['home.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats.loc[:,'away.possessionTime'] = stats['away.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats['home.win'] = (stats['home.points'] > stats['away.points']).astype(int)
stats['away.win'] = (stats['away.points'] > stats['home.points']).astype(int)
stats.iloc[1239,22] = 10
stats = stats.drop(9284, axis = 0)



/Users/yihaoy/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yihaoy/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/yihaoy/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [28]:
away_index = [x for x in list(stats.columns) if 'away.' in x] + ['Week', 'Year', 'id']
away = stats[away_index].drop('away.points', axis = 1)
away.columns = [x.split('.')[-1] for x in list(away.columns)]
away['isHome'] = 0
away = away.groupby(['Year', 'school','Week']).mean()
away

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2004 Air Force        4         0.375000                14.0           9.9   
                      5         0.523810                16.0          10.6   
                      10        0.538462                21.0           9.0   
                      11        0.416667                 8.0          10.5   
     Akron            2         0.384615                36.0           6.2   
...                                  ...                 ...           ...   
2020 West Virginia    8         0.266667                50.0           6.9   
     Western Kentucky 2         0.333333                23.0           5.6   
                      5         0.411765                36.0           5.2   
                      7         0.083333                35.0           4.6   
     Wyoming          8         0.187500                32.0           7.3   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2004 Air Force        4                     3.9                  0.0   
                      5                     4.6                  0.0   
                      10                    4.8                  0.0   
                      11                    5.2                  0.0   
     Akron            2                     2.3                  0.0   
...                                         ...                  ...   
2020 West Virginia    8                     3.1                 69.0   
     Western Kentucky 2                     3.4                 23.0   
                      5                     3.6                 55.0   
                      7                     2.8                 39.0   
     Wyoming          8                     3.7                  6.0   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2004 Air Force        4           0.0          1454.0             71.0   
                      5           3.0          1273.0             66.0   
                      10          2.0           783.0             48.0   
                      11          0.0          1184.0             66.0   
     Akron            2           3.0          1034.0             31.0   
...                               ...             ...              ...   
2020 West Virginia    8           1.0          1765.0             29.0   
     Western Kentucky 2           1.0          1647.0             35.0   
                      5           1.0          1895.0             38.0   
                      7           4.0          1713.0             32.0   
     Wyoming          8           1.0          1502.0             35.0   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2004 Air Force        4          17.0             5.058824        4.905882   
                      5         -14.0             7.318841        5.743902   
                      10        -17.0             6.352273        6.086957   
                      11          9.0             5.878378        5.797297   
     Akron            2         -38.0             8.515625        4.402985   
...                               ...                  ...             ...   
2020 West Virginia    8          -7.0             5.194030        5.544304   
     Western Kentucky 2         -14.0             6.763889        4.275862   
                      5           3.0             4.833333        4.405405   
                      7         -23.0             5.424242        3.731343   
     Wyoming          8          -3.0             6.280000        5.388060   

                            forcedTO  passAttemptsAllowed  \
Year school           Week           

In [29]:
home_index = [x for x in list(stats.columns) if 'home.' in x] + ['Week', 'Year', 'id']
home = stats[home_index].drop('home.points',axis = 1)
home.columns = [x.split('.')[-1] for x in list(home.columns)]
home['isHome'] = 1
home = home.groupby(['Year', 'school','Week']).mean()

In [30]:
total = pd.concat([home,away]).groupby(['Year', 'school','Week']).mean()
total.iloc[-10:,:]

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2020 West Virginia    7         0.375000                44.0           7.2   
                      8         0.266667                50.0           6.9   
     Western Kentucky 2         0.333333                23.0           5.6   
                      3         0.300000                25.0           7.7   
                      5         0.411765                36.0           5.2   
                      6         0.166667                39.0           5.4   
                      7         0.083333                35.0           4.6   
                      8         0.250000                25.0           5.3   
     Wisconsin        8         0.571429                21.0          11.8   
     Wyoming          8         0.187500                32.0           7.3   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2020 West Virginia    7                     5.9                 76.0   
                      8                     3.1                 69.0   
     Western Kentucky 2                     3.4                 23.0   
                      3                     3.5                 19.0   
                      5                     3.6                 55.0   
                      6                     3.4                  0.0   
                      7                     2.8                 39.0   
                      8                     3.7                 25.0   
     Wisconsin        8                     3.4                 15.0   
     Wyoming          8                     3.7                  6.0   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2020 West Virginia    7           2.0          1975.0             38.0   
                      8           1.0          1765.0             29.0   
     Western Kentucky 2           1.0          1647.0             35.0   
                      3           0.0          1501.0             28.0   
                      5           1.0          1895.0             38.0   
                      6           3.0          1519.0             25.0   
                      7           4.0          1713.0             32.0   
                      8           2.0          1615.0             33.0   
     Wisconsin        8           1.0          2608.0             54.0   
     Wyoming          8           1.0          1502.0             35.0   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2020 West Virginia    7          21.0             2.907407        6.634146   
                      8          -7.0             5.194030        5.544304   
     Western Kentucky 2         -14.0             6.763889        4.275862   
                      3          -6.0             6.164557        5.490566   
                      5           3.0             4.833333        4.405405   
                      6         -24.0             5.548387        4.593750   
                      7         -23.0             5.424242        3.731343   
                      8           3.0             3.461538        4.379310   
     Wisconsin        8          38.0             4.804348        5.733333   
     Wyoming          8          -3.0             6.280000        5.388060   

                            forcedTO  passAttemptsAllowed  \
Year school           Week                                  
2020 West Virginia    7          2.0                 23.0   
                      8          1.0                 28.0   
     Western Kentucky 2          2.0                 34.0   
                      3          0.0                 21.0   
                      5          0

In [31]:
idx = list(total.index)
idx.reverse()
n = 1
ids = total['id']
field_status = total['isHome']
outcomes = total['win']
total.drop(['id','isHome'], axis = 1, inplace = True) 
todrop = []
for i,x in enumerate(idx[:-1]):
    if(x[1] != idx[i + 1][1]):
        todrop.append(x)
        continue
    j = 2
    agg = total.loc[idx[i+1],:]
    while((i + j < len(idx)) and x[1] == idx[i + j][1]):
        agg = agg + total.loc[idx[i+j],:]
        j += 1
        n += 1
    total.loc[x,:] = agg/n
    n = 1;
total['id'] = ids
total['isHome'] = field_status
total['won_game'] = outcomes
total.drop(todrop, axis = 0, inplace = True)

In [32]:
# need to drop the rows in todrop, put stats back in matchup df from unique gameids in total
rowlist = []
for gameID in stats['id']:
    if len(total[total['id'] == gameID]) != 2: continue
    hometeam = list((total[(total['id'] == gameID) & (total['isHome'] == 1)].values)[0])
    awayteam = list((total[(total['id'] == gameID) & (total['isHome'] == 0)].values)[0])
    rowlist.append(hometeam + awayteam)
    rowlist.append(awayteam + hometeam)
matchups = pd.DataFrame(rowlist, columns = (list(total.columns) + ['other.' + x for x in list(total.columns)])).drop(['other.id','other.isHome','other.won_game'], axis = 1)
matchups = matchups.set_index('id')
matchups['win_pct_diff'] = matchups['win'] - matchups['other.win']
matchups.drop(['win', 'other.win'], axis = 1, inplace = True)
outcome = matchups['won_game']
matchups.drop('won_game', axis = 1, inplace = True)
matchups['outcome'] = outcome

In [33]:
matchups.tail(20)
matchups.columns

Index(['thirdDownEff', 'completionAttempts', 'yardsPerPass',
       'yardsPerRushAttempt', 'totalPenaltiesYards', 'turnovers',
       'possessionTime', 'rushingAttempts', 'pointDiff', 'yardsAllowedPerPlay',
       'yards_per_play', 'forcedTO', 'passAttemptsAllowed',
       'yardsAllowedPerRush', 'isHome', 'other.thirdDownEff',
       'other.completionAttempts', 'other.yardsPerPass',
       'other.yardsPerRushAttempt', 'other.totalPenaltiesYards',
       'other.turnovers', 'other.possessionTime', 'other.rushingAttempts',
       'other.pointDiff', 'other.yardsAllowedPerPlay', 'other.yards_per_play',
       'other.forcedTO', 'other.passAttemptsAllowed',
       'other.yardsAllowedPerRush', 'win_pct_diff', 'outcome'],
      dtype='object')

In [34]:
# import everything
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Try Regression models for score differential- Lasso, Ridge

## Data processing

In [35]:
total = pd.concat([home,away]).groupby(['Year', 'school','Week']).mean()
total.iloc[-10:,:]

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2020 West Virginia    7         0.375000                44.0           7.2   
                      8         0.266667                50.0           6.9   
     Western Kentucky 2         0.333333                23.0           5.6   
                      3         0.300000                25.0           7.7   
                      5         0.411765                36.0           5.2   
                      6         0.166667                39.0           5.4   
                      7         0.083333                35.0           4.6   
                      8         0.250000                25.0           5.3   
     Wisconsin        8         0.571429                21.0          11.8   
     Wyoming          8         0.187500                32.0           7.3   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2020 West Virginia    7                     5.9                 76.0   
                      8                     3.1                 69.0   
     Western Kentucky 2                     3.4                 23.0   
                      3                     3.5                 19.0   
                      5                     3.6                 55.0   
                      6                     3.4                  0.0   
                      7                     2.8                 39.0   
                      8                     3.7                 25.0   
     Wisconsin        8                     3.4                 15.0   
     Wyoming          8                     3.7                  6.0   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2020 West Virginia    7           2.0          1975.0             38.0   
                      8           1.0          1765.0             29.0   
     Western Kentucky 2           1.0          1647.0             35.0   
                      3           0.0          1501.0             28.0   
                      5           1.0          1895.0             38.0   
                      6           3.0          1519.0             25.0   
                      7           4.0          1713.0             32.0   
                      8           2.0          1615.0             33.0   
     Wisconsin        8           1.0          2608.0             54.0   
     Wyoming          8           1.0          1502.0             35.0   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2020 West Virginia    7          21.0             2.907407        6.634146   
                      8          -7.0             5.194030        5.544304   
     Western Kentucky 2         -14.0             6.763889        4.275862   
                      3          -6.0             6.164557        5.490566   
                      5           3.0             4.833333        4.405405   
                      6         -24.0             5.548387        4.593750   
                      7         -23.0             5.424242        3.731343   
                      8           3.0             3.461538        4.379310   
     Wisconsin        8          38.0             4.804348        5.733333   
     Wyoming          8          -3.0             6.280000        5.388060   

                            forcedTO  passAttemptsAllowed  \
Year school           Week                                  
2020 West Virginia    7          2.0                 23.0   
                      8          1.0                 28.0   
     Western Kentucky 2          2.0                 34.0   
                      3          0.0                 21.0   
                      5          0

In [36]:
idx = list(total.index)
idx.reverse()
n = 1
ids = total['id']
field_status = total['isHome']
scoreDiff = total['pointDiff']
total.drop(['id','isHome'], axis = 1, inplace = True) 
todrop = []
for i,x in enumerate(idx[:-1]):
    if(x[1] != idx[i + 1][1]):
        todrop.append(x)
        continue
    j = 2
    agg = total.loc[idx[i+1],:]
    while((i + j < len(idx)) and x[1] == idx[i + j][1]):
        agg = agg + total.loc[idx[i+j],:]
        j += 1
        n += 1
    total.loc[x,:] = agg/n
    n = 1;
total['id'] = ids
total['isHome'] = field_status
total['scoreDiff'] = scoreDiff
total.drop(todrop, axis = 0, inplace = True)

In [37]:
total.tail(10)

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2020 Wake Forest      8         0.431349               30.50      9.700000   
     West Virginia    4         0.615385               35.00      8.400000   
                      5         0.484163               36.00      8.050000   
                      7         0.415368               38.00      7.033333   
                      8         0.405276               39.50      7.075000   
     Western Kentucky 3         0.333333               23.00      5.600000   
                      5         0.316667               24.00      6.650000   
                      6         0.348366               28.00      6.166667   
                      7         0.302941               30.75      5.975000   
                      8         0.259020               31.60      5.700000   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2020 Wake Forest      8                3.675000            45.000000   
     West Virginia    4                6.500000            85.000000   
                      5                4.050000            95.500000   
                      7                3.833333            97.666667   
                      8                4.350000            92.250000   
     Western Kentucky 3                3.400000            23.000000   
                      5                3.450000            21.000000   
                      6                3.500000            32.333333   
                      7                3.475000            24.250000   
                      8                3.340000            27.200000   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2020 Wake Forest      8      0.250000     1589.500000        42.250000   
     West Virginia    4      0.000000     2143.000000        51.000000   
                      5      1.000000     2038.000000        46.500000   
                      7      2.000000     1970.333333        44.000000   
                      8      2.000000     1971.500000        42.500000   
     Western Kentucky 3      1.000000     1647.000000        35.000000   
                      5      0.500000     1574.000000        31.500000   
                      6      0.666667     1681.000000        33.666667   
                      7      1.250000     1640.500000        31.500000   
                      8      1.800000     1655.000000        31.600000   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2020 Wake Forest      8     10.500000             5.784016        6.208004   
     West Virginia    4     46.000000             4.204082        7.255814   
                      5     16.000000             4.816327        5.855755   
                      7     12.666667             4.412762        5.323590   
                      8     14.750000             4.036424        5.651229   
     Western Kentucky 3    -14.000000             6.763889        4.275862   
                      5    -10.000000             6.464223        4.883214   
                      6     -5.666667             5.920593        4.723945   
                      7    -10.250000             5.827542        4.691396   
                      8    -12.800000             5.746882        4.499385   

                            forcedTO  passAttemptsAllowed  \
Year school           Week                                  
2020 Wake Forest      8     2.250000            35.250000   
     West Virginia    4     1.000000            17.000000   
                      5     1.000000            19.500000   
                      7     1.333333            25.666667   
                      8     1.5000

In [38]:
# need to drop the rows in todrop, put stats back in matchup df from unique gameids in total
rowlist = []
for gameID in stats['id']:
    if len(total[total['id'] == gameID]) != 2: continue
    hometeam = list((total[(total['id'] == gameID) & (total['isHome'] == 1)].values)[0])
    awayteam = list((total[(total['id'] == gameID) & (total['isHome'] == 0)].values)[0])
    rowlist.append(hometeam + awayteam)
    rowlist.append(awayteam + hometeam)
matchups = pd.DataFrame(rowlist, columns = (list(total.columns) + ['other.' + x for x in list(total.columns)])).drop(['other.id','other.isHome','other.scoreDiff'], axis = 1)
matchups = matchups.set_index('id')

matchups['win_pct_diff'] = matchups['win'] - matchups['other.win']
matchups.drop(['win', 'other.win'], axis = 1, inplace = True)
# here I did not add point diff diff feature 
scoreDiff = matchups['scoreDiff']
matchups.drop('scoreDiff', axis = 1, inplace = True)
matchups['scoreDiff'] = scoreDiff

## Split data

In [39]:
from sklearn.model_selection import train_test_split

matchups = matchups.astype(float).dropna()
X, y = matchups.iloc[:,:-1], matchups.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    random_state = 53)


## Basic Lasso Regression Model

In [40]:
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error

lasso = Lasso(alpha = 0.001, normalize=True)
lasso.fit(X_train, y_train)

lasso.score(X_train, y_train)

0.3331688893282334

In [41]:
mean_squared_error(lasso.predict(X_train), y_train)

326.8964933485013

In [42]:
lasso.coef_

array([ 0.00000000e+00,  8.63755124e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.37496970e-01, -0.00000000e+00,  1.43827376e+00, -5.59442996e-01,
        0.00000000e+00, -1.30744817e+00,  9.09365472e+00,  0.00000000e+00,
       -6.51288818e-03, -1.28338763e-01, -0.00000000e+00,  1.55417994e-03,
       -5.20001879e-02,  0.00000000e+00,  0.00000000e+00, -3.42563841e-01,
        1.76455028e-01, -1.18021693e+00,  5.36239522e-01, -5.63281233e-03,
        1.15886821e+00,  5.73633013e+00])

## Lasso Regression CV

In [43]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)


LassoCV(cv=10, max_iter=100000, normalize=True)

In [44]:
lassocv.alpha_

0.0005691304529144445

In [45]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
lasso.score(X_train, y_train)

0.3336132759407435

In [46]:
lasso.score(X_test, y_test)

0.33291605575008854

In [47]:
pred_test = lasso.predict(X_test)
pred_test = np.sign(pred_test)
accuracy_score(pred_test, np.sign(y_test))

0.7076894433066468

## Lasso CV - accuracy as metric

In [74]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer

def computeAcc(y,y_pred):
    return accuracy_score(np.sign(y),np.sign(y_pred))

accScorer = make_scorer(computeAcc, greater_is_better=True)
parameters = {"alpha": 0.0005*np.logspace(-3,3,num = 500,base=10)}

grid = GridSearchCV(lasso, param_grid=parameters,
                   scoring = accScorer)


In [75]:
grid.fit(X_train, y_train)


GridSearchCV(estimator=Lasso(alpha=0.007281742387506222, normalize=True),
             param_grid={'alpha': array([5.00000000e-07, 5.14036612e-07, 5.28467277e-07, 5.43303057e-07,
       5.58555325e-07, 5.74235774e-07, 5.90356423e-07, 6.06929631e-07,
       6.23968103e-07, 6.41484899e-07, 6.59493448e-07, 6.78007555e-07,
       6.97041413e-07, 7.16609613e-07, 7.36727155e-07, 7.57409461e-07,
       7.78672386e...
       2.64494654e-01, 2.71919871e-01, 2.79553539e-01, 2.87401508e-01,
       2.95469795e-01, 3.03764584e-01, 3.12292236e-01, 3.21059285e-01,
       3.30072455e-01, 3.39338653e-01, 3.48864982e-01, 3.58658747e-01,
       3.68727455e-01, 3.79078823e-01, 3.89720788e-01, 4.00661506e-01,
       4.11909367e-01, 4.23472990e-01, 4.35361242e-01, 4.47583236e-01,
       4.60148340e-01, 4.73066188e-01, 4.86346681e-01, 5.00000000e-01])},
             scoring=make_scorer(computeAcc))

In [76]:
best_alpha = grid.best_params_['alpha']
best_alpha


0.007858918588865806

In [77]:
lasso.set_params(alpha=best_alpha)
lasso.fit(X_train, y_train)
lasso.score(X_train, y_train)

0.3220343539060314

In [78]:
pred_test = lasso.predict(X_test)
pred_test = np.sign(pred_test)
accuracy_score(pred_test, np.sign(y_test))

0.7102960342580525

## Ridge Regression

In [48]:
alphas = 10**np.linspace(10,-2,100)*0.5
ridgecv = RidgeCV(alphas=alphas, cv = 10, normalize=True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

0.005

In [49]:
ridge = Ridge(alpha=ridgecv.alpha_)
ridge.fit(X_train, y_train)
ridge.score(X_train, y_train)

0.3339919217059568

In [50]:
pred_test = ridge.predict(X_test)
pred_test = np.sign(pred_test)
accuracy_score(pred_test, np.sign(y_test))

0.7061999627629864

## Ridge CV - accuracy as metric

In [84]:
parameters = {"alpha": 0.005*np.logspace(-3,3,num = 500,base=10)}
grid = GridSearchCV(ridge, param_grid=parameters,
                   scoring = accScorer)
grid.fit(X_train, y_train)

GridSearchCV(estimator=Ridge(alpha=0.8042347416326776),
             param_grid={'alpha': array([5.00000000e-06, 5.14036612e-06, 5.28467277e-06, 5.43303057e-06,
       5.58555325e-06, 5.74235774e-06, 5.90356423e-06, 6.06929631e-06,
       6.23968103e-06, 6.41484899e-06, 6.59493448e-06, 6.78007555e-06,
       6.97041413e-06, 7.16609613e-06, 7.36727155e-06, 7.57409461e-06,
       7.78672386e-06, 8.00532231e-0...
       2.64494654e+00, 2.71919871e+00, 2.79553539e+00, 2.87401508e+00,
       2.95469795e+00, 3.03764584e+00, 3.12292236e+00, 3.21059285e+00,
       3.30072455e+00, 3.39338653e+00, 3.48864982e+00, 3.58658747e+00,
       3.68727455e+00, 3.79078823e+00, 3.89720788e+00, 4.00661506e+00,
       4.11909367e+00, 4.23472990e+00, 4.35361242e+00, 4.47583236e+00,
       4.60148340e+00, 4.73066188e+00, 4.86346681e+00, 5.00000000e+00])},
             scoring=make_scorer(computeAcc))

In [85]:
best_alpha = grid.best_params_['alpha']
best_alpha

0.8042347416326776

In [86]:
ridge = Ridge(alpha=best_alpha)
ridge.fit(X_train, y_train)
ridge.score(X_train, y_train)

0.3339919006595584

In [87]:
pred_test = ridge.predict(X_test)
pred_test = np.sign(pred_test)
accuracy_score(pred_test, np.sign(y_test))

0.7061999627629864